In [1]:
!pip install qiskit[visualization]==1.1.0
!pip install qiskit_aer
!pip install qiskit_ibm_runtime
!pip install matplotlib
!pip install pylatexenc
!pip install prototype-zne
#%pip install git+https://github.com/ryanhill1/Quantum-Challenge-Grader.git


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
%set_env QXToken=ee6a8eeaba4e73fb88220e80cf1ae551b1444e2aea7f6c1c2cc0038aeb24a66f464f96de618f1b057931b21600f3bc71b34b3d37d3258fd215872f4bb93edede

env: QXToken=ee6a8eeaba4e73fb88220e80cf1ae551b1444e2aea7f6c1c2cc0038aeb24a66f464f96de618f1b057931b21600f3bc71b34b3d37d3258fd215872f4bb93edede


In [3]:
from qc_grader.challenges.qgss_2024 import *

from math import pi
from qiskit.circuit.library import QFT
from qiskit.providers.fake_provider import GenericBackendV2, generic_backend_v2
generic_backend_v2._NOISE_DEFAULTS["cx"] = (5.99988e-06, 6.99988e-06, 1e-5, 5e-3)

from qiskit import transpile, QuantumCircuit
from qiskit.circuit import Gate
from qiskit.converters import circuit_to_dag
from qiskit.transpiler import CouplingMap, StagedPassManager, PassManager, AnalysisPass, TransformationPass
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler.preset_passmanagers.common import generate_unroll_3q, generate_embed_passmanager
from qiskit.quantum_info import hellinger_fidelity
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.dagcircuit import DAGCircuit
from qiskit_ibm_runtime.fake_provider import FakeTorino

# Transpiler Passes
## Layout passes
from qiskit.transpiler.passes.layout.csp_layout import CSPLayout
from qiskit.transpiler.passes.layout.dense_layout import DenseLayout
from qiskit.transpiler.passes.layout.sabre_layout import SabreLayout
from qiskit.transpiler.passes.layout.vf2_layout import VF2Layout
from qiskit.transpiler.passes.layout.trivial_layout import TrivialLayout

## Routing passes
from qiskit.transpiler.passes.routing.basic_swap import BasicSwap
from qiskit.transpiler.passes.routing.lookahead_swap import LookaheadSwap
from qiskit.transpiler.passes.routing.sabre_swap import SabreSwap
from qiskit.transpiler.passes.routing.stochastic_swap import StochasticSwap
from qiskit.transpiler.passes.routing.star_prerouting import StarPreRouting

## Synthesis passes (passes for the translation stage)
from qiskit.circuit import SessionEquivalenceLibrary
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary
from qiskit.transpiler.passes.basis.basis_translator import BasisTranslator
from qiskit.transpiler.passes.synthesis.high_level_synthesis import HighLevelSynthesis
### The next pass could also be considered an optimization pass.
from qiskit.transpiler.passes.synthesis.unitary_synthesis import UnitarySynthesis

## Optimization passes
from qiskit.transpiler.passes.optimization.collect_1q_runs import Collect1qRuns
from qiskit.transpiler.passes.optimization.collect_2q_blocks import Collect2qBlocks
from qiskit.transpiler.passes.optimization.consolidate_blocks import ConsolidateBlocks
from qiskit.transpiler.passes.optimization.commutative_cancellation import CommutativeCancellation

In [4]:
num_qubits = 10
qc = QFT(num_qubits, do_swaps=False)
qc.draw()

┌──────┐
q_0: ┤0     ├
     │      │
q_1: ┤1     ├
     │      │
q_2: ┤2     ├
     │      │
q_3: ┤3     ├
     │      │
q_4: ┤4     ├
     │  QFT │
q_5: ┤5     ├
     │      │
q_6: ┤6     ├
     │      │
q_7: ┤7     ├
     │      │
q_8: ┤8     ├
     │      │
q_9: ┤9     ├
     └──────┘

In [5]:
def get_qc_characteristics(qc):
    
    depth = qc.depth()
    num_qubits = qc.num_qubits
    ops = qc.count_ops()
    num_multi_qubit_ops = qc.num_nonlocal_gates()
    
    return {"depth": depth, "num_qubits": num_qubits, "ops": ops, "num_multi_qubit_ops": num_multi_qubit_ops}

In [6]:
grade_lab1_ex1(get_qc_characteristics)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [7]:
def print_qc_characteristics(qc):
    characteristics = get_qc_characteristics(qc)
    print("Quantum circuit characteristics")
    print("  Depth:", characteristics['depth'])
    print("  Number of qubits:", characteristics['num_qubits'])
    print("  Operations:", dict(characteristics['ops']))
    print("  Number of multi-qubit Operations:", characteristics['num_multi_qubit_ops'])

print_qc_characteristics(qc)

Quantum circuit characteristics
  Depth: 1
  Number of qubits: 10
  Operations: {'QFT': 1}
  Number of multi-qubit Operations: 1


In [8]:
qc_dec = qc.decompose()
get_qc_characteristics(qc_dec)
qc_dec.draw(fold=-1)

┌───┐
q_0: ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────────────────────────■──────────────────────────────────────■──────────────────────────────────■───────────────────────────■───────────────────────■──────────────────■─────────────■────────■───────┤ H ├
                                                                                                                                                                                                                                                                     │                                            │                                      │                                  │                           │                       │                  │       ┌───┐ │        │P(π/2) └───┘
q_1: ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────────■──────────┼──────────────────────────────────■─────────┼────────────────────────────■─────────┼────────────────────────■─────────┼──────────────────■────────┼──────────────■────────┼─────────■────────┼───────┤ H ├─┼────────■────────────
                                                                                                                                                                                                                   │                                      │          │                                  │         │                            │         │                        │         │                  │        │        ┌───┐ │        │         │P(π/2)  │       └───┘ │P(π/4)               
q_2: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────────────■─────────┼────────────────────────────■─────────┼──────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼───────────────■────────┼─────────┼─────────■────────┼────────┼────────┤ H ├─┼────────┼─────────■────────┼─────────────■─────────────────────
                                                                                                                                                                      │                                  │         │                            │         │          │                        │         │         │                   │        │         │         ┌───┐ │        │         │         │P(π/2)  │        │        └───┘ │P(π/4)  │                  │P(π/8)                             
q_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────■─────────┼────────────────────────■─────────┼─────────┼───────────────────■────────┼─────────┼──────────┼───────────────■────────┼─────────┼─────────┼──────────■────────┼────────┼─────────┼─────────┤ H ├─┼────────┼─────────┼─────────■────────┼────────┼──────────────■────────┼──────────────────■───────────────────────────────────
                                                                                                                                │                           │         │                        │         │         │                   │        │         │          │         ┌───┐ │        │         │         │          │P(π/2)  │        │         │         └───┘ │P(π/4)  │         │                  │P(π/8)  │                       │P(π/16)                                               
q_4: ─────────────────────────────────────────────────────────────────────────────────────

In [9]:
backend = GenericBackendV2(num_qubits)
print("Supported basis gates:", backend.operation_names)

Supported basis gates: ['cx', 'id', 'rz', 'sx', 'x', 'reset', 'delay', 'measure']


In [10]:
qc_synth = generate_preset_pass_manager(2, backend=backend).run(qc)
qc_synth.draw(fold=-1)

global phase: 0
         ┌─────────┐┌────┐┌────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
q_9 -> 0 ┤ Rz(π/2) ├┤ √X ├┤ Rz(3.1385) ├──■────────────────■───────■────────────────────────■────────────────────■─────────────────────────────■───────■────────────────────────■─────────────────────────■─────────────────────────■────────────────────■───────────────────────────■─────────────────■─────────────────────────────■─────────────────■─────────────────────────────────────────────────■──────────────────────■─────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         └─────────┘└────┘└────────────┘  │                │       │                        │                    │                             │     ┌─┴─┐    ┌───────────┐   ┌─┴─┐     ┌──────────┐      │                  ┌───┐  │  ┌───────────┐     │      ┌───┐ ┌──────────┐   │      ┌───┐      │    ┌──────────┐      ┌───┐  │   ┌─────────┐   │                ┌───┐┌──────────┐                │               ┌───┐  │   ┌──────────┐    ┌────┐    ┌────────────┐      │                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
q_5 -> 1 ─────────────────────────────────┼────────────────┼───────┼────────────────────────┼────────────────────┼─────────────────────────────┼─────┤ X ├────┤ Rz(-π/32) ├───┤ X ├─────┤ Rz(π/32) ├──────┼──────────────────┤ X ├──┼──┤ Rz(-π/16) ├─────┼──────┤ X ├─┤ Rz(π/16) ├───┼──────┤ X ├──────┼────┤ Rz(-π/8) ├──────┤ X ├──┼───┤ Rz(π/8) ├───┼────────────────┤ X ├┤ Rz(-π/4) ├────────────────┼───────────────┤ X ├──┼───┤ Rz(3π/4) ├────┤ √X ├────┤ Rz(3.0925) ├──────┼─────────────────────────────────────────────────────────■────────────────────────────────────────────■────────────────────────■─────────────────────■─────────────────────────────────────────■──────────────────────────────────■──────────────────────────────■───────────────────────────────■────────────────■────────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [11]:
print_qc_characteristics(qc_synth)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [12]:
# Your work goes here!
# assign a 10-qubit linear `CouplingMap` to the variable cm
cm = CouplingMap([[i, i+1] for i in range(9)])

# add your transpiled quantum circuit to the next line
qc_routed = transpile(qc, basis_gates=backend.operation_names, coupling_map=cm)

In [13]:
grade_lab1_ex2(qc_routed)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [52]:
pm_staged = StagedPassManager()
pm_staged.init = generate_unroll_3q(None)
pm_staged.layout = PassManager()

pm_staged.layout += TrivialLayout(cm)

pm_staged.layout += generate_embed_passmanager(cm)

In [64]:
pm_staged.routing = PassManager(LookaheadSwap(coupling_map=cm)) 
qc_routed = pm_staged.run(qc)
print_qc_characteristics(qc_routed)
qc_routed.draw(fold=-1)

Quantum circuit characteristics
  Depth: 309
  Number of qubits: 10
  Operations: {'cx': 456, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 456


global phase: 1.5723
                    ┌───┐                                                                                                                                                                                                                                                                                                                       ┌───┐     ┌─────────┐                                                                                                                     ┌──────────┐                                                       ┌──────────┐                                                                                                                                                                                                                                                                                                                                                                  ┌───┐                                                                                                                                          ┌───┐                                                                        ┌───┐                                                                                                                                                                                                  ┌───┐                                                       ┌───┐                     ┌───┐              ┌───┐          ┌───┐          ┌───┐                                                                                                                                                                                                                                                                                                                                                         ┌───┐                                                                                            ┌───┐                                                                                                         ┌───┐                                                                                                                          ┌───┐                       ┌───┐                        ┌───┐                                                                                                                                                                                                                                                                                                                               
q_0 -> 0 ─────■─────┤ X ├──────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├──■──┤ Rz(π/8) ├────────────────────────────────────────────────────────────────────────────────────■─────────────────────────■──────┤ Rz(π/16) ├─────────────────────■──────────────────────────■──────┤ Rz(π/32) ├─────────────────■─────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────┤ X ├────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├──────■──────────────────────────────────────────────────────────■──────┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────┤ X ├─────────■──────────────────────────────────────■──────┤ X ├──────■────────■─────┤ X ├──────■────■──┤ X ├──■────■──┤ X ├─

In [65]:
grade_lab1_ex3(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [66]:
pm_staged.translation = PassManager([
    HighLevelSynthesis(),
    BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names)
])
qc_routed_synth = pm_staged.run(qc)
print_qc_characteristics(qc_routed_synth)
qc_routed_synth.draw(fold=-1)

Quantum circuit characteristics
  Depth: 309
  Number of qubits: 10
  Operations: {'cx': 456, 'rz': 155, 'sx': 10}
  Number of multi-qubit Operations: 456


global phase: 1.5723
                    ┌───┐                                                                                                                                                                                                                                                                                                                       ┌───┐     ┌─────────┐                                                                                                                     ┌──────────┐                                                       ┌──────────┐                                                                                                                                                                                                                                                                                                                                                                  ┌───┐                                                                                                                                          ┌───┐                                                                        ┌───┐                                                                                                                                                                                                  ┌───┐                                                       ┌───┐                     ┌───┐              ┌───┐          ┌───┐          ┌───┐                                                                                                                                                                                                                                                                                                                                                         ┌───┐                                                                                            ┌───┐                                                                                                         ┌───┐                                                                                                                          ┌───┐                       ┌───┐                        ┌───┐                                                                                                                                                                                                                                                                                                                               
q_0 -> 0 ─────■─────┤ X ├──────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├──■──┤ Rz(π/8) ├────────────────────────────────────────────────────────────────────────────────────■─────────────────────────■──────┤ Rz(π/16) ├─────────────────────■──────────────────────────■──────┤ Rz(π/32) ├─────────────────■─────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────┤ X ├────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├──────■──────────────────────────────────────────────────────────■──────┤ X ├──■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────┤ X ├─────────■──────────────────────────────────────■──────┤ X ├──────■────────■─────┤ X ├──────■────■──┤ X ├──■────■──┤ X ├─

In [67]:
grade_lab1_ex4(pm_staged)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [68]:
qk_qc = generate_preset_pass_manager(2, backend=backend).run(qc)
print_qc_characteristics(qk_qc)

Quantum circuit characteristics
  Depth: 65
  Number of qubits: 10
  Operations: {'rz': 101, 'cx': 90, 'sx': 10}
  Number of multi-qubit Operations: 90


In [69]:
def noisy_sim(qc, backend):
    
    return backend.run(qc.measure_all(inplace=False), shots=7 * 1024).result().get_counts()

own_transpiler_sim = noisy_sim(qc_routed_synth, backend)
qiskit_transpiler_sim = noisy_sim(qk_qc, backend)
reference_sim = noisy_sim(transpile(qc.decompose(), backend=backend), BasicSimulator())

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.1123
Qiskit transpiler fidelity 0.5389


In [303]:
pm_opt = StagedPassManager()

pm_opt.init = generate_unroll_3q(None)

pm_opt.layout = PassManager()

pm_opt.layout += TrivialLayout(cm)

pm_opt.routing = PassManager(StarPreRouting())

pm_opt.translation = PassManager([
    BasisTranslator(equivalence_library=SessionEquivalenceLibrary, target_basis=backend.operation_names)
])

pm_opt.optimization = PassManager([
    Collect2qBlocks(),
    ConsolidateBlocks(),
    UnitarySynthesis(basis_gates=backend.operation_names),
    BasisTranslator(SessionEquivalenceLibrary, target_basis=backend.operation_names)
])

pm_opt.scheduling = None

pm_opt.layout += generate_embed_passmanager(cm)

qc_opt = pm_opt.run(qc)

In [280]:
grade_lab1_ex5(pm_opt)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [305]:
print_qc_characteristics(qc_opt)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_opt)['num_multi_qubit_ops']/get_qc_characteristics(qk_qc)['num_multi_qubit_ops']), 3)
print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
qc_opt.draw(fold=-1)

Quantum circuit characteristics
  Depth: 180
  Number of qubits: 10
  Operations: {'rz': 399, 'sx': 182, 'cx': 133, 'x': 75}
  Number of multi-qubit Operations: 133
Reduction in two-qubit gates compared to qiskit -47.778%!


global phase: 0.001534
          ┌──────────┐┌────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               ┌───┐     ┌──────────┐               ┌───┐    ┌────────────┐    ┌───┐      ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
q_0 -> 0 ─┤ Rz(-π/4) ├┤ √X ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├─────┤ Rz(-π/2) ├───────────────┤ X ├────┤ Rz(1.5677) ├────┤ X ├──────┤ Rz(π/4) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [306]:
opt_transpiler_sim = noisy_sim(qc_opt, backend)

print("Own transpiler fidelity", round(hellinger_fidelity(own_transpiler_sim, reference_sim), 4))
print("Qiskit transpiler fidelity", round(hellinger_fidelity(qiskit_transpiler_sim, reference_sim), 4))
print("Own optimized transpiler fidelity", round(hellinger_fidelity(opt_transpiler_sim, reference_sim), 4))

Own transpiler fidelity 0.1123
Qiskit transpiler fidelity 0.5389
Own optimized transpiler fidelity 0.9047


In [314]:
num_qubits = 133
basis_gates = ['u1', 'u2', 'u3', 'cx', 'id', 'rz', 'sx', 'x', 'measure']  
coupling_map = CouplingMap.from_full(num_qubits)  

target_backend = None

In [313]:
num_qubits = 100
qc = QFT(num_qubits, do_swaps=False)
target_backend = None

pm_student = StagedPassManager()

pm_student.init = generate_unroll_3q(None)
pm_student.layout = PassManager()
pm_student.layout += TrivialLayout(coupling_map)
pm_student.routing = PassManager(StarPreRouting())
pm_student.translation = PassManager([
    BasisTranslator(SessionEquivalenceLibrary, target_basis=backend.operation_names)
])
pm_student.optimization = PassManager([
    Collect2qBlocks(),
    ConsolidateBlocks(),
    UnitarySynthesis(basis_gates=basis_gates),
    BasisTranslator(SessionEquivalenceLibrary, target_basis=backend.operation_names)
])
pm_student.scheduling = None
pm_student.layout += generate_embed_passmanager(coupling_map)

qc_student = pm_student.run(qc)
qc_qiskit = generate_preset_pass_manager(2, backend=target_backend).run(qc)
reduction_ratio = round(100-100*(get_qc_characteristics(qc_student)['num_multi_qubit_ops']/get_qc_characteristics(qc_qiskit)['num_multi_qubit_ops']), 3)

print("Reduction in two-qubit gates compared to qiskit {}%!".format(reduction_ratio))
print("N-qubit gates generated by Qiskit's default pass manager:", get_qc_characteristics(qc_qiskit)['num_multi_qubit_ops'])
print("N-qubit gates generated by your pass manager", get_qc_characteristics(qc_student)['num_multi_qubit_ops'])

Reduction in two-qubit gates compared to qiskit -199.96%!
N-qubit gates generated by Qiskit's default pass manager: 4950
N-qubit gates generated by your pass manager 14848


In [317]:
class GatesPerQubit(AnalysisPass):
    def __init__(self):
        super().__init__()

    def run(self, dag: DAGCircuit):
        one_q_op = {}
        two_q_op = {}

        for qubit in dag.qubits:
            one_q_op[qubit] = 0
            two_q_op[qubit] = 0
        
        for node in dag.op_nodes():
            if len(node.qargs) == 1:  
                qubit = node.qargs[0]
                one_q_op[qubit] += 1
            elif len(node.qargs) == 2:  
                for qubit in node.qargs:
                    two_q_op[qubit] += 1
 
        self.property_set['one_q_op'] = one_q_op
        self.property_set['two_q_op'] = two_q_op


In [316]:
grade_lab1_ex6(GatesPerQubit)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.


In [318]:
qc = QFT(4, do_swaps=False)
qc = generate_preset_pass_manager(2, backend=backend).run(qc)
gpq = GatesPerQubit()
gpq(qc)
print("single-qubit gates on qubit", gpq.property_set["one_q_op"])
print("two-qubit gates on qubit", gpq.property_set["two_q_op"])
qc.draw(fold=-1, idle_wires=False)

single-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 0): 0, Qubit(QuantumRegister(10, 'q'), 1): 0, Qubit(QuantumRegister(10, 'q'), 2): 4, Qubit(QuantumRegister(10, 'q'), 3): 0, Qubit(QuantumRegister(10, 'q'), 4): 6, Qubit(QuantumRegister(10, 'q'), 5): 0, Qubit(QuantumRegister(10, 'q'), 6): 0, Qubit(QuantumRegister(10, 'q'), 7): 3, Qubit(QuantumRegister(10, 'q'), 8): 0, Qubit(QuantumRegister(10, 'q'), 9): 8}
two-qubit gates on qubit {Qubit(QuantumRegister(10, 'q'), 0): 0, Qubit(QuantumRegister(10, 'q'), 1): 0, Qubit(QuantumRegister(10, 'q'), 2): 6, Qubit(QuantumRegister(10, 'q'), 3): 0, Qubit(QuantumRegister(10, 'q'), 4): 6, Qubit(QuantumRegister(10, 'q'), 5): 0, Qubit(QuantumRegister(10, 'q'), 6): 0, Qubit(QuantumRegister(10, 'q'), 7): 6, Qubit(QuantumRegister(10, 'q'), 8): 0, Qubit(QuantumRegister(10, 'q'), 9): 6}


global phase: 0
                                        ┌───┐┌──────────┐┌───┐┌──────────┐   ┌────┐   ┌──────────┐                                                                                                                                                            
q_2 -> 2 ───────────────────────────────┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├───┤ √X ├───┤ Rz(7π/8) ├─────────────────■───────────────────────■────────────────────■────────────────────────────■────────────────────────────────────────────────────────────────
                                        └─┬─┘└──────────┘└─┬─┘└──┬───┬───┘┌──┴────┴──┐└──┬───┬───┘┌─────────┐    ┌─┴─┐    ┌──────────┐   ┌─┴─┐    ┌──────────┐  │     ┌────┐   ┌──────────┐  │                                                                
q_1 -> 4 ─────────────────────────────────┼────────────────┼─────┤ X ├────┤ Rz(-π/8) ├───┤ X ├────┤ Rz(π/8) ├────┤ X ├────┤ Rz(-π/4) ├───┤ X ├────┤ Rz(3π/4) ├──┼─────┤ √X ├───┤ Rz(3π/4) ├──┼───────────────■────────────────■───────────────────────────────
         ┌─────────┐┌────┐┌────────────┐  │                │     └─┬─┘    └──────────┘   └─┬─┘    └─────────┘    └───┘    └──────────┘   └───┘    └──────────┘  │     └────┘   └──────────┘  │               │                │                               
q_3 -> 7 ┤ Rz(π/2) ├┤ √X ├┤ Rz(15π/16) ├──■────────────────■───────■───────────────────────■───────────■───────────────────────■────────────────────────────────┼────────────────────────────┼───────────────┼────────────────┼───────────────────────────────
         └─────────┘└────┘└────────────┘                                                             ┌─┴─┐   ┌───────────┐   ┌─┴─┐    ┌──────────┐            ┌─┴─┐┌──────────┐            ┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐┌────┐┌─────────┐
q_0 -> 9 ────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───┤ Rz(-π/16) ├───┤ X ├────┤ Rz(π/16) ├────────────┤ X ├┤ Rz(-π/8) ├────────────┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(3π/4) ├┤ √X ├┤ Rz(π/2) ├
                                                                                                     └───┘   └───────────┘   └───┘    └──────────┘            └───┘└──────────┘            └───┘└─────────┘└───┘└──────────┘└───┘└──────────┘└────┘└─────────┘

In [353]:
pg = Gate('Peres', 3, params=[], label='PG')

In [354]:
qc_pg = QuantumCircuit(3)
qc_pg.append(pg, [0, 1, 2])
qc_pg.draw()

┌─────┐
q_0: ┤0    ├
     │     │
q_1: ┤1 PG ├
     │     │
q_2: ┤2    ├
     └─────┘

In [366]:
def get_qc_in(nq):
    qc_qft = QFT(nq, do_swaps=False)
    qc_inner = QuantumCircuit(nq)
    for i in range(1, nq - 1):
        qc_inner.append(pg, [nq - i - 2, nq - i - 1, nq - 1])

    qc_in = QuantumCircuit(nq)
    qc_in.compose(qc_qft, range(nq), inplace=True)

    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.rz(pi, nq - 1)
    qc_in.compose(qc_inner, range(nq), inplace=True)

    for i in range(nq // 2):
        qc_in.cx(i, nq - i - 1)
        qc_in.cx(nq - i - 1, i)
        qc_in.cx(i, nq - i - 1)

    qc_in.compose(qc_qft.inverse(), range(nq), inplace=True)
    return qc_in

nq = 5
qc_in = get_qc_in(nq)
qc_in_dag = circuit_to_dag(qc_in) 
qc_in.draw(fold=-1)

┌──────┐          ┌───┐                                 ┌─────┐          ┌───┐               ┌───────┐
q_0: ┤0     ├──■───────┤ X ├───────■─────────────────────────┤0    ├──■───────┤ X ├───────■───────┤0      ├
     │      │  │       └─┬─┘┌───┐  │                  ┌─────┐│     │  │       └─┬─┘┌───┐  │       │       │
q_1: ┤1     ├──┼────■────┼──┤ X ├──┼──────■───────────┤0    ├┤1    ├──┼────■────┼──┤ X ├──┼────■──┤1      ├
     │      │  │    │    │  └─┬─┘  │      │    ┌─────┐│     ││     │  │    │    │  └─┬─┘  │    │  │       │
q_2: ┤2 QFT ├──┼────┼────┼────┼────┼──────┼────┤0    ├┤1    ├┤  PG ├──┼────┼────┼────┼────┼────┼──┤2 IQFT ├
     │      │  │  ┌─┴─┐  │    │    │    ┌─┴─┐  │     ││  PG ││     │  │  ┌─┴─┐  │    │    │  ┌─┴─┐│       │
q_3: ┤3     ├──┼──┤ X ├──┼────■────┼────┤ X ├──┤1 PG ├┤     ├┤     ├──┼──┤ X ├──┼────■────┼──┤ X ├┤3      ├
     │      │┌─┴─┐└───┘  │       ┌─┴─┐┌─┴───┴─┐│     ││     ││     │┌─┴─┐└───┘  │       ┌─┴─┐└───┘│       │
q_4: ┤4     ├┤ X ├───────■───────┤ X ├┤ Rz(π) ├┤2    ├┤2    ├┤2    ├┤ X ├───────■───────┤ X ├─────┤4      ├
     └──────┘└───┘               └───┘└───────┘└─────┘└─────┘└─────┘└───┘               └───┘     └───────┘

In [387]:
class PeresGateTranslation(TransformationPass):
    def get_peres_decomposition(self):
        qcsx = QuantumCircuit(2)
        qcsx.rz(pi / 4, 0)
        qcsx.rz(pi / 2, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)
        qcsx.cx(0, 1)
        qcsx.rz(-pi / 4, 1)
        qcsx.cx(0, 1)
        qcsx.rz(3 * pi / 4, 1)
        qcsx.sx(1)
        qcsx.rz(pi / 2, 1)

        qcsx_inv = QuantumCircuit(2)
        qcsx_inv.rz(pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 0)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 2, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.sx(1)
        qcsx_inv.rz(-3 * pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(-pi / 4, 1)
        qcsx_inv.cx(0, 1)
        qcsx_inv.rz(pi / 4, 0)

        qc_dec = QuantumCircuit(3)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(1, 0)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.compose(qcsx_inv, [1, 2], inplace=True)
        qc_dec.cx(0, 1)
        qc_dec.cx(0, 1)
        return qc_dec

    def run(self, dag: DAGCircuit):
        peres_gate_name = "Peres"
        peres_decomposition = self.get_peres_decomposition()

        for node in dag.named_nodes(peres_gate_name):
            dag.substitute_node_with_dag(node,circuit_to_dag(peres_decomposition))

        return dag 


In [388]:
grade_lab1_ex7(PeresGateTranslation, pg)

Submitting your answer. Please wait...
Congratulations 🎉! Your answer is correct and has been submitted.
